In [32]:
import torch
import numpy as np
import torch.nn as nn
import math
import os
import pandas as pd
import torch.optim as optim
import torch.nn.functional as F
import json
import copy
from torch import nn, Tensor

D_MODEL = 252
PADDING_TOKEN = 0
UNK_TOKEN = 1
SOS_TOKEN = 2
EOS_TOKEN = 3
MAX_LENGTH = 200
tokens = json.load(open("tokens.txt"))
model = torch.load("embedding_model")
file_path = os.path.join(os.path.expanduser("~"), "Downloads", "mental_health.csv")
orig_dataset = pd.read_csv(file_path)
orig_dataset = orig_dataset.to_numpy()
unfiltered_trainingSet =  orig_dataset[np.random.choice(orig_dataset.shape[0], 100, replace=True)] #extract training set
trainingSet = []
for example in unfiltered_trainingSet:
    if type(example[0]) is not float and type(example[1]) is not float:
        if len(example[0].split())<200 and len(example[1].split())<200:
            trainingSet.append(example)

            
contextSet = [trainingSet[i][0] for i in range(len(trainingSet))] 
responseSet = [trainingSet[i][1] for i in range(len(trainingSet))]
#tokenizing the context and response set, also 1 is special token for unknown word
contextSet_tokenized = [[tokens[word] if word in tokens else UNK_TOKEN for word in example.split()] 
                         for example in contextSet ]



responseSet_tokenized = [[tokens[word] if word in tokens else UNK_TOKEN for word in example.split()] 
                         for example in responseSet ]

label =copy.deepcopy(responseSet_tokenized)
#set up special tokens
for i in range(len(contextSet_tokenized)):
    while len(contextSet_tokenized[i])!= MAX_LENGTH:
        contextSet_tokenized[i].append(PADDING_TOKEN)
for i in range(len(responseSet_tokenized)):
    responseSet_tokenized[i].insert(0, SOS_TOKEN)
    label[i].append(EOS_TOKEN)
for i in range(len(responseSet_tokenized)):
    while len(responseSet_tokenized[i]) != MAX_LENGTH:
        responseSet_tokenized[i].append(PADDING_TOKEN)
        label[i].append(PADDING_TOKEN)

    
#convert token to input embedding for context and response set filled with padding
#tokens if end of sentence. 0 is special token for padding 
contextSet_embedding = []        
for context in contextSet_tokenized:
    contextEmbedding = []
    for i in range(MAX_LENGTH):
        if i>= len(context):
            contextEmbedding.append(model["embeddings.weight"][1])
            continue
        contextEmbedding.append(model["embeddings.weight"][context[i]])
    contextEmbedding = torch.stack(contextEmbedding)
    contextSet_embedding.append(contextEmbedding[:])

contextSet_embedding = torch.stack(contextSet_embedding)
responseSet_embedding = []        
for response in responseSet_tokenized:
    responseEmbedding = []
    for i in range(MAX_LENGTH):
        responseEmbedding.append(model["embeddings.weight"][response[i]])
    responseEmbedding = torch.stack(responseEmbedding)
    responseSet_embedding.append(responseEmbedding[:])

contextSet_tokenized = torch.tensor(contextSet_tokenized)
responseSet_tokenized = torch.tensor(responseSet_tokenized)

responseSet_embedding = torch.stack(responseSet_embedding)
print(responseSet_embedding.shape)


torch.Size([67, 200, 252])


In [3]:
class PositionalEncoding(nn.Module):
    def __init__(self, d_model = D_MODEL, seq_len = MAX_LENGTH, dropout = 0.1) -> None:
        super().__init__()
        self.d_model = d_model
        self.seq_len = seq_len
        self.dropout = nn.Dropout(dropout)
        # Create a matrix of shape (seq_len, d_model)
        pe = torch.zeros(seq_len, d_model)
        # Create a vector of shape (seq_len)
        position = torch.arange(0, seq_len, dtype=torch.float).unsqueeze(1) # (seq_len, 1)
        # Create a vector of shape (d_model)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model)) # (d_model / 2)
        # Apply sine to even indices
        pe[:, 0::2] = torch.sin(position * div_term) # sin(position * (10000 ** (2i / d_model))
        # Apply cosine to odd indices
        pe[:, 1::2] = torch.cos(position * div_term) # cos(position * (10000 ** (2i / d_model))
        # Add a batch dimension to the positional encoding
        pe = pe.unsqueeze(0) # (1, seq_len, d_model)
        # Register the positional encoding as a buffer
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + (self.pe[:, :x.shape[1], :]).requires_grad_(False) # (batch, seq_len, d_model)
        return self.dropout(x)


In [4]:
class MultiHeadAttentionBlock(nn.Module):
    def __init__(self, d_model: int, h: int, dropout: float)->None:
        super().__init__()
        self.d_model = d_model # Embedding vector size
        self.h = h # Number of heads
        # Make sure d_model is divisible by h
        assert d_model % h == 0, "d_model is not divisible by h"
        self.d_k = d_model // h # Dimension of vector seen by each head
        self.w_q = nn.Linear(d_model, d_model, bias=False) # linear layer for queue
        self.w_k = nn.Linear(d_model, d_model, bias=False) # liner layer for key
        self.w_v = nn.Linear(d_model, d_model, bias=False) # linear layer for 
        self.w_o = nn.Linear(d_model, d_model, bias=False) # Wo
        self.dropout = nn.Dropout(dropout)
                 
    @staticmethod
    def attention(query, key, value, mask, dropout: nn.Dropout):
        #d_k is number of embeddings per word for each head
        d_k = query.shape[-1]
        # Just apply the formula from the paper
        # (batch, h, seq_len, d_k) --> (batch, h, seq_len, seq_len)
        # [[[[4,3],[2,1]],[[2, 5], [3,2]]]] * [[[[4,3][2,5]],[[2,1], [3,2]]]] = 
        #[[[[0.7, 0.3], [0.1, 0.9]], [0.8, 0.2], [0.4, 0.6]]]
        #basically saying the first word of the first half of the embedding(for head 1) is related to
        #to its own word by 0.7 and related to the first half embedding of the second word by 0.3
        #also saying the second word of the first half of the embedding(for head 1) is related to
        #to its own word by 0.9 and related to the first half embedding of the first word by 0.1
        #essentially a attention matrix for each part of the embedding for the head
        attention_scores = (query @ key.transpose(-2, -1)) / math.sqrt(d_k)
        #if mask is true, then make all appropriate masked attentions scores to very low value
        #so that the soft max will ignore their scores
        if mask is not None:
            # Write a very low value (indicating -inf) to the positions where mask == 0
            attention_scores.masked_fill_(mask == 0, -1e9)
        attention_scores = attention_scores.softmax(dim=-1) # (batch, h, seq_len, seq_len) # Apply softmax
        if dropout is not None:
            attention_scores = dropout(attention_scores)
        # (batch, h, seq_len, seq_len) --> (batch, h, seq_len, d_k)
        # return attention scores which can be used for visualization
        return (attention_scores @ value), attention_scores
    
    def forward(self, q, k, v, mask):
        query = self.w_q(q) # (batch, seq_len, d_model) --> (batch, seq_len, d_model)
        key = self.w_k(k) # (batch, seq_len, d_model) --> (batch, seq_len, d_model)
        value = self.w_v(v) # (batch, seq_len, d_model) --> (batch, seq_len, d_model)
        # (batch, seq_len, d_model) --> (batch, seq_len, h, d_k) --> (batch, h, seq_len, d_k)
        #[[[4,3,2,1] , [2,5,3,2]]]
        #3rd dimension represents word embeddings of a word
        #2nd dimension represent each word embedding in an example
        #1st dimension represents each example containing each word embedding
        # * Transform the dimensions using .view *
        # [[[[4,3],[2,1]],[[2, 5], [3,2]]]]
        #4th dimenension represents the size of the split embeddings for each multi-head attention
        #3rd dimension represents the number of heads. Note that head times the size of split embeddings equals original embedding dimension
        #2nd dimension represents sequence length
        #1st dimension represents example lenght
        # * Transponse dimensions 1,2 *
        # [[[[4,3][2,5]],[[2,1], [3,2]]]]
        # 4th dimension represents the size of the split embeddings for each multi-head attention
        # 3rd dimension represents the length of the sequence. Contains each word with its associated head
        # 2nd dimension represents the number of heads
        # 1st dimension is the number of examples
        #[[[[word 1 with first half embeddings][word 2 with first half embedding]],[[word 1 with second half], [word 2 with second half]]]]        
        query = query.view(query.shape[0], query.shape[1], self.h, self.d_k).transpose(1, 2)
        key = key.view(key.shape[0], key.shape[1], self.h, self.d_k).transpose(1, 2)
        value = value.view(value.shape[0], value.shape[1], self.h, self.d_k).transpose(1, 2)
        # Calculate attention
        x, self.attention_scores = MultiHeadAttentionBlock.attention(query, key, value, mask, self.dropout)
        # Combine all the heads together
        # (batch, h, seq_len, d_k) --> (batch, seq_len, h, d_k) --> (batch, seq_len, d_model)
        x = x.transpose(1, 2).contiguous().view(x.shape[0], -1, self.h * self.d_k)
        #pass combined heads to last linear layer
        return self.w_o(x)



In [5]:
class FeedForwardBlock(nn.Module):
    def __init__(self, d_model: int, d_ff: int, dropout: float) -> None:
        super().__init__()
        self.linear_1 = nn.Linear(d_model, d_ff) # w1 and b1
        self.dropout = nn.Dropout(dropout)
        self.linear_2 = nn.Linear(d_ff, d_model) # w2 and b2

    def forward(self, x):
        # (batch, seq_len, d_model) --> (batch, seq_len, d_ff) --> (batch, seq_len, d_model)
        #the output of the self.linear1 layer is the input of a relu layer
        #the output of that is the input of linear layer 2
     
        return self.linear_2(self.dropout(torch.relu(self.linear_1(x))))

In [6]:
#normalizing the data for stabilized training
class LayerNormalization(nn.Module):
    def __init__(self, features: int, eps:float=10**-6) -> None:
        super().__init__()
        self.eps = eps
        self.alpha = nn.Parameter(torch.ones(features)) # alpha is a learnable parameter
        self.bias = nn.Parameter(torch.zeros(features)) # bias is a learnable parameter

    def forward(self, x):
        # x: (batch, seq_len, hidden_size)
         # Keep the dimension for broadcasting
        mean = x.mean(dim = -1, keepdim = True) # (batch, seq_len, 1)
        # Keep the dimension for broadcasting
        std = x.std(dim = -1, keepdim = True) # (batch, seq_len, 1)
        # eps is to prevent dividing by zero or when std is very small
        return self.alpha * (x - mean) / (std + self.eps) + self.bias


In [7]:
class ResidualConnection(nn.Module):
    def __init__(self, features: int, dropout: float) -> None:
        super().__init__()
        self.dropout = nn.Dropout(dropout)
        self.norm = LayerNormalization(features)
    def forward(self, x, sublayer):
        # in the paper, you would typically add x to the sublayer and then normalize the output
        #of the sublayer, but in this case, we normalize before passing it in to the sublayer
        #the .norm part is the normalizing part in ADD and NORM, and the addition of x is the 
        #ADD part of ADD and NORM
        return x + self.dropout(sublayer(self.norm(x)))


In [8]:
class EncoderBlock(nn.Module):

    def __init__(self, features: int, self_attention_block: MultiHeadAttentionBlock, feed_forward_block: FeedForwardBlock, dropout: float) -> None:
        super().__init__()
        self.self_attention_block = self_attention_block
        self.feed_forward_block = feed_forward_block
        #encoder block contains 2 residual connections
        self.residual_connections = nn.ModuleList([ResidualConnection(features, dropout) for _ in range(2)])

    def forward(self, x, src_mask):
        #Passes self attention block to residuals. Residuals will perform the attention block
        #and also perform the ADD and NORM
        x = self.residual_connections[0](x, lambda x: self.self_attention_block(x, x, x, src_mask))
        #passes the output of the previous residual_connection layer('x'). Then, passes in a feed_forward_block
        #Residual connection will perform the feed forward and ADD and NORM
        x = self.residual_connections[1](x, self.feed_forward_block)
        return x
    

In [9]:
class Encoder(nn.Module):
    def __init__(self, features: int, layers: nn.ModuleList) -> None:
        super().__init__()
        #layers of encoder blocks
        self.layers = layers
        self.norm = LayerNormalization(features)

    def forward(self, x, mask):
        #perform a forward method on every encoder block. The output of each encoder
        #becomes the input of the new encoder
        for layer in self.layers:
            x = layer(x, mask)
        #normalize the final output
        return self.norm(x)

In [10]:
class DecoderBlock(nn.Module):
    def __init__(self, features: int, self_attention_block: MultiHeadAttentionBlock, cross_attention_block: MultiHeadAttentionBlock, feed_forward_block: FeedForwardBlock, dropout: float) -> None:
        super().__init__()
        self.self_attention_block = self_attention_block
        self.cross_attention_block = cross_attention_block
        self.feed_forward_block = feed_forward_block
        self.residual_connections = nn.ModuleList([ResidualConnection(features, dropout) for _ in range(3)])
    
    #src mask are for the encoder output. Do not want to attend to padding tokens
    #tgt mask are for decoder input. Doesn't let you look into the future
    #typically these just always be true I think
    def forward(self, x, encoder_output, src_mask, tgt_mask):
        x = self.residual_connections[0](x, lambda x: self.self_attention_block(x, x, x, tgt_mask))
        x = self.residual_connections[1](x, lambda x: self.cross_attention_block(x, encoder_output, encoder_output, src_mask))
        x = self.residual_connections[2](x, self.feed_forward_block)
        return x

In [11]:
class Decoder(nn.Module):
    def __init__(self, features: int, layers: nn.ModuleList) -> None:
        super().__init__()
        self.layers = layers
        self.norm = LayerNormalization(features)
    def forward(self, x, encoder_output, src_mask, tgt_mask):
        for layer in self.layers:
            x = layer(x, encoder_output, src_mask, tgt_mask)
        return self.norm(x)

In [12]:
#linear layer where output represents all words. We are going to soft max this 
#in the future. 
class ProjectionLayer(nn.Module):
    def __init__(self, d_model, vocab_size) -> None:
        super().__init__()
        self.proj = nn.Linear(d_model, vocab_size)

    def forward(self, x) -> None:
        # (batch, seq_len, d_model) --> (batch, seq_len, vocab_size)
        return self.proj(x)
    

In [23]:
class Transformer(nn.Module):
    def __init__(self, encoder: Encoder, decoder: Decoder, src_embed, tgt_embed, src_pos: PositionalEncoding, tgt_pos: PositionalEncoding, projection_layer: ProjectionLayer, src_pad_indx = 0, device = "cpu") -> None:
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.src_embed = src_embed
        self.tgt_embed = tgt_embed
        self.src_pos = src_pos
        self.tgt_pos = tgt_pos
        self.projection_layer = projection_layer
        self.src_pad_idx = src_pad_indx
        self._pad_idx = src_pad_indx
        self.device = device
    
    def make_src_mask(self, src_tokens):
        src_mask = (src_tokens != self.src_pad_idx).unsqueeze(1).unsqueeze(2)
        # (N, 1, 1, src_len)
        return src_mask.to(self.device)
    
    def make_trg_mask(self, trg_tokens):
        N, trg_len = trg_tokens.shape
        trg_mask = torch.tril(torch.ones((trg_len, trg_len))).expand(
            N, 1, trg_len, trg_len
        )
        return trg_mask.to(self.device)

    def encode(self, src, src_mask):
        # (batch, seq_len, d_model)
        
        src = self.src_pos(src)
        return self.encoder(src, src_mask)
    
    def decode(self, encoder_output: torch.Tensor, src_mask: torch.Tensor, tgt: torch.Tensor, tgt_mask: torch.Tensor):
        # (batch, seq_len, d_model)
        tgt = self.tgt_pos(tgt)
        return self.decoder(tgt, encoder_output, src_mask, tgt_mask)
    
    def project(self, x):
        # (batch, seq_len, vocab_size)
        return self.projection_layer(x)

In [25]:
def build_transformer(src_embed, tgt_embed, src_vocab_size: int, tgt_vocab_size: int, src_seq_len: int, tgt_seq_len: int, d_model: int=512, N: int=6, h: int=8, dropout: float=0.1, d_ff: int=2048, device = "cpu") -> Transformer:

    # Create the positional encoding layers
    src_pos = PositionalEncoding(d_model, src_seq_len, dropout).to(device)
    tgt_pos = PositionalEncoding(d_model, tgt_seq_len, dropout).to(device)
    

    # Create the encoder blocks
    encoder_blocks = []
    for _ in range(N):
        encoder_self_attention_block = MultiHeadAttentionBlock(d_model, h, dropout).to(device)
        feed_forward_block = FeedForwardBlock(d_model, d_ff, dropout).to(device)
        encoder_block = EncoderBlock(d_model, encoder_self_attention_block, feed_forward_block, dropout).to(device)
        encoder_blocks.append(encoder_block)

    # Create the decoder blocks
    decoder_blocks = []
    for _ in range(N):
        decoder_self_attention_block = MultiHeadAttentionBlock(d_model, h, dropout).to(device)
        decoder_cross_attention_block = MultiHeadAttentionBlock(d_model, h, dropout).to(device)
        feed_forward_block = FeedForwardBlock(d_model, d_ff, dropout).to(device)
        decoder_block = DecoderBlock(d_model, decoder_self_attention_block, decoder_cross_attention_block, feed_forward_block, dropout).to(device)
        decoder_blocks.append(decoder_block)
    
    # Create the encoder and decoder
    encoder = Encoder(d_model, nn.ModuleList(encoder_blocks)).to(device)
    decoder = Decoder(d_model, nn.ModuleList(decoder_blocks)).to(device)
    
    # Create the projection layer
    projection_layer = ProjectionLayer(d_model, tgt_vocab_size).to(device)
    
    # Create the transformer
    transformer = Transformer(encoder, decoder, src_embed, tgt_embed, src_pos, tgt_pos, projection_layer, device=device).to(device)
    
    # Initialize the parameters
    for p in transformer.parameters():
        if p.dim() > 1:
            nn.init.xavier_uniform_(p)
    
    return transformer

In [31]:
device = "mps" if torch.backends.mps.is_available() else "cpu"
transformer = build_transformer(contextSet_embedding, responseSet_embedding, src_vocab_size = 4604, 
                                tgt_vocab_size = 4604, src_seq_len = MAX_LENGTH, 
                                tgt_seq_len = MAX_LENGTH, d_model = 252, N =6, h=6, dropout=0.1, device=device).to(device)
label = torch.tensor(label).to(device)
# Loss and optimizer
criterion = nn.CrossEntropyLoss().to(device)
optimizer = optim.Adam(transformer.parameters(), eps=1e-9)
num_epochs = 100
for epoch in range(num_epochs):
    print(epoch)
    losses = 0
    decoder_mask = transformer.make_src_mask(contextSet_tokenized).to(device)
    encoder_mask = transformer.make_trg_mask(responseSet_tokenized).to(device)
    encoder_input = contextSet_embedding.to(device)
    decoder_input = responseSet_embedding.to(device)
    encoder_output = transformer.encode(encoder_input, encoder_mask) # (B, seq_len, d_model)
    decoder_output = transformer.decode(encoder_output, encoder_mask, decoder_input, decoder_mask) # (B, seq_len, d_model)
    proj_output = transformer.project(decoder_output) # (B, seq_len, vocab_size)
    loss = criterion(proj_output.view(-1, 4604), label.view(-1))
    loss.backward()

    # Update the weights
    optimizer.step()
    optimizer.zero_grad(set_to_none=True)
    losses += loss
    print(losses)
    #Inside your loop
    with torch.no_grad():  # Disable gradient calculation for inference
        logits = proj_output  # Assuming proj_output is your logits tensor
        probabilities = F.softmax(logits, dim=-1)  # Apply softmax along the last dimension
        predicted_classes = torch.argmax(probabilities, dim=-1)  # Find the index of the class with the highest probability

        # Convert tensors to numpy arrays for inspection
        logits_array = logits.numpy()
        probabilities_array = probabilities.numpy()
        predicted_classes_array = predicted_classes.numpy()        
        sent = ""
        for i in predicted_classes_array[0]:
            if i<5:
                continue
            for word in tokens:
                if tokens[word] == i:
                    sent = sent + word
                    sent = sent + " "
        print("")
        print("True Response: ")
        print("")
        print(responseSet[0])
        print("")
        print("Predicted Response: ")
        print("")
        print(sent)
            


0


RuntimeError: MPS backend out of memory (MPS allocated: 20.38 GB, other allocations: 35.86 MB, max allowed: 20.40 GB). Tried to allocate 7.25 KB on private pool. Use PYTORCH_MPS_HIGH_WATERMARK_RATIO=0.0 to disable upper limit for memory allocations (may cause system failure).